#Correcting telluric absorption with ${\tt molecfit}$
*Elyar Sedaghati*

Typically, when analyzing astronomical spectra, the imprint of absorption and emission from the earth's atmosphere can have a significant impact on the analysis being performed. This is especially true in high reoslution transmission spectroscopy of exoplanetary atmospheres. The telluric features can introduce spurious signals when performing cross correlation analysis with atmospheric templates (see the lecture by Matteo Brogi & Jens Hoeijmakers), or perfoming narrowband tranmission spectroscopy on individual strong transition lines (see the lecture by Julia Seidel & Núria Casasayas-Baris). Therefore it is essential to obtain a precise transmission model for the earth's atmosphere above the observatory.

![Telluric absorption](https://drive.google.com/uc?export=view&id=1YKSXbR9oIBuSJ8oLS7NshX17X_sdaXTU)

One widely used software that takes an empirical approach to this problem is called ${\tt molecfit}$ which was initially an in-kind contribution to ESO from Austria and is currently being maintained and further developed by ESO. In a nutshell it obtains the transmission model by solving a line-by-line radiative transfer equation, utilizing the atmospheric profile above the observatory. In general, it fits for the pressure profile, the initial value of which it gets from the humidity and temperature profiles that it obtains from the GDAS database, through the hypsometric relation. 

## Installation
Installing ${\tt molecfit}$ has never been too straighforward unfortunately.  Previously, it was available as a standalone software with binary distributions, which are now deprecated, but can still be found on an ESO ftp server:

https://ftp.eso.org/pub/dfs/pipelines/skytools/molecfit/

However, ESO no longer offers support for these versions, as a major overhaul has migrated the running of ${\tt molecfit}$ exclusively to ${\tt esoreflex}$.  This is an environment where all ESO reduction/analysis workflows are executed:

https://www.eso.org/sci/software/esoreflex/

To install the latest version of ${\tt molecfit}$ together with ${\tt esoreflex}$ please follow the instructions on ESO's website:


*   For Mac users: https://www.eso.org/sci/software/pipelines/installation/macports.html
*   For Lunix users: https://www.eso.org/sci/software/pipelines/installation/rpm.html

I myself use a Mac, and by far the easiest way to install has been with **macports**.

##Running ${\tt molecfit}$ (deprecated binary version)
I will first go through running the standalone version of ${\tt molecfit}$ from the command line. This is for you to get familiar with various parameters used to fit for the atmospheric transmission function and correcting the spectra using that solution.

For that we first have to clone the directory with all the necessary materials:


In [ ]:
!rm -r molecfit_lecture/
!git clone https://github.com/esedagha/molecfit_lecture
!unzip molecfit_lecture/output_demo.zip
!mv output_demo molecfit_lecture/output_demo/
!rm -r __MACOSX/

In this directory there is a python file called *createParFile* that is a wrapper for creating the parameter file that drives molecfit routines.  This wrapper is based on https://github.com/ryanhoughton/PyMolecFit. We need to import it here:

In [ ]:
from molecfit_lecture.createParFile import createParFile

Some important points to consider before running molecfit:

1.   (For now) ${\tt molecfit}$ only runs on 1D spectra. Namely, you cannot feed a 2D array from individual orders of an echellegram to it.
2.   The wavelength solution needs to be in the observatory frame. Namely, no barycentric motion correction should be applied. This seems to no longer be an issue for the new version.

Now lets have a look at the example spectrum that is included in the directory.  This is a spectrum of a GV star taken with ESPRESSO at the VLT, which is a high resolution, cross-dispersed, echelle spectrograph in the coudé lab at Cerro Paranal.

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt

#hdr = fits.open('molecfit_lecture/ESPRESSO_spectrum.fits')[0].header #if you want to inspect the header elements. 
hdu = fits.open('molecfit_lecture/ESPRESSO_spectrum.fits')[1].data
print(hdu.columns)

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(15,10))
ax1.plot(hdu['WAVELENGTH'], hdu['FLUX'], lw=0.1)
ax2.set_ylabel('Flux', fontsize=20)
ax2.plot(hdu['WAVELENGTH'], hdu['FLUX'], lw=1, label=r'H$_2$O absorption band')
ax2.set_xlim(7200,7400)
ax2.set_ylim(-50,2400)
ax2.legend(fontsize=16)
ax3.plot(hdu['WAVELENGTH'], hdu['FLUX'], lw=1, label=r'O$_2$ absorption band')
ax3.set_xlim(7580,7720)
ax3.set_ylim(-50,2000)
ax3.legend(fontsize=16)
ax3.set_xlabel(r'Wavelength [$\AA$]', fontsize=20)

By far the trickiest part of correcting spectra with the standalone version of ${\tt molecfit}$ is selecting **only** the telluric absorption lines, as well as regions of continuum **only**. Namely, we need to feed to molecfit only telluric lines that are not blended with any other non-atmospheric lines (stellar or ISM for instance). This is particularly difficult to achieve for cooler stars where many lines are present.

The selection of regions to be used for fitting is done via 2 text files:

1.   include.dat
2.   exclude.dat

We will use the include file to give molecfit the general regions where the fitting will be done.  This should include at least 1 region for each species to be fit.  The exclude file is then used to define regions within those include ranges that are to be excluded from fitting; namely stellar/ISM lines.  You should also always avoid using very deep (saturated) telluric lines, as there is very little information is retained in them. Also note that the wavelengths in these files should be give in $\mu m$.

To make the selection, I typically overplot the spectrum with a previsouly dervied transmission model to find out isolated telluric lines and continuum regions.

In the *molecfit_lecture* directory an exemplar set of include and exclude files has been included.  Here we plot them for you to see how the regions are selected:

*note: I created those files by capturing my clicks on an interactive matplotlib window, which somewhat speeds up the tedious process.*

In [ ]:
import numpy as np
include   = np.loadtxt('molecfit_lecture/include.dat')
exclude   = np.loadtxt('molecfit_lecture/exclude.dat')
tel       = np.loadtxt('molecfit_lecture/Telluric_transmission.ascii')

fig, axes = plt.subplots(len(include),1, figsize=(15,10))
for i,ranges in enumerate(include):
  axes[i].plot(tel[:,0], tel[:,1]*2000., ls='--', lw=1, alpha=0.7, c='k', label='Generic Transmission model' if i==0 else '')
  axes[i].plot(hdu['WAVELENGTH'], hdu['FLUX'], lw=1, label='Stellar Spectrum' if i==0 else '')
  axes[i].set_xlim(10000*ranges[0],10000*ranges[1])

for e in exclude:
  wvl = hdu['WAVELENGTH']
  for i in range(len(include)):
    axes[i].plot(wvl[np.logical_and(wvl>10000*e[0],wvl<10000*e[1])], hdu['FLUX'][np.logical_and(wvl>10000*e[0],wvl<10000*e[1])],\
                 lw=1, c='r', label='Masked regions' if i==0 and e[0]==exclude[0,0] else '')

axes[0].legend(loc='lower left', fontsize=12, framealpha=0.5)
axes[3].set_xlabel(r'Wavelength [$\AA$]', fontsize=16)

Now that we have our include and exclude regions, we can create the parameter file for driving molecfit. This we will do with the help of the *createParFile* function that we imported earlier.

In [ ]:
!ls molecfit_lecture/

createParFile.py	include.dat	 Telluric_transmission.ascii
ESPRESSO_spectrum.fits	output_demo
exclude.dat		output_demo.zip


In [ ]:
root_dir = '/content/molecfit_lecture/'
infile   = 'ESPRESSO_spectrum.fits' 
hdr      = fits.open(root_dir+infile)[0].header
data     = fits.open(root_dir+infile)[1].data 

createParFile(infile,\
              parfile_dir=root_dir,\
              obj_dir=root_dir,\
              gdas_prof='',gdas_dir='~/molecfit/share/molecfit/data/profiles/gdas',\
              outdir=root_dir+infile[:infile.rfind(".fits")],\
              outfile=infile[:infile.rfind(".fits")],\
              wrange_include=root_dir+'include.dat',\
              wrange_exclude=root_dir+'exclude.dat',\
              columns=["WAVELENGTH", "FLUX", "FLUX_ERR", "NULL"],\
              ftol=1e-6,xtol=1e-6,\
              list_molec=['H2O', 'O2'], fit_molec=[True, True,],\
              fit_cont=True, cont_n=4, cont_const=np.percentile(data.FLUX,90),\
              fit_wlc=True, wlc_n=1, wlc_const=-0.0, \
              fitresbox=False, kernmode=True,\
              fit_res_box=False, relres_box=0.0,\
              fit_res_gauss=True, res_gauss=3.0,\
              fit_res_lorentz=True, res_lorentz=1.5,\
              kernfac=30.0, varkern=True, kernel_file=None,\
              telalt_key= hdr['ESO TEL* ALT'].cards[0][0],\
              rhum_key=hdr['ESO TEL* AMBI RHUM'].cards[0][0], \
              pres_key=hdr['ESO TEL* AMBI PRES START'].cards[0][0], \
              temp_key=hdr['ESO TEL* AMBI TEMP'].cards[0][0], \
              m1temp_key=hdr['ESO TEL* TH M1 TEMP'].cards[0][0], \
              geoelev_key=hdr['ESO TEL* GEOELEV'].cards[0][0], \
              longitude_key=hdr['ESO TEL* GEOLON'].cards[0][0], \
              latitude_key=hdr['ESO TEL* GEOLAT'].cards[0][0])

Some notes about the parameters:

*   ${\tt columns}$: should match what is in the fits file
*   ${\tt ftol}$ & ${\tt xtol}$: determine the $\chi^2$ and parameter conversion criterion.  For actual analysis set to 1e-8 or 1e-10
*   ${\tt list\_molec}$: # species to be fit. These will increase when you move to NIR (e.g. CH$_4$, CO$_2$, etc.)
*   ${\tt cont}$: fitting the continuum and degree of the local polynomial.
*   ${\tt wlc}$: using the telluric lines to fine-tune the wavelength solution.
*   ${\tt kern}$: kernel parameters for fitting line shapes.



In [ ]:
!cat molecfit_lecture/ESPRESSO_spectrum.par

Now we are ready to finally run molecfit, which entails running two separate routines, ${\tt molecfit}$ and ${\tt calctrans}$.  You run them simply by typing these commands in the terminal.

``` $> ~/molecfit/bin/molecfit ESPRESSO_spectrum.par m ```

``` $> ~/molecfit/bin/calctrans ESPRESSO_spectrum.par ```

Once both routines have finished running (the run time depends on the $\chi^2$ tolerance paramter), all your results will be saved in the output directory we defined in the ${\tt createParFile}$ function, and was created when we ran it.

I have run this on my own machine, and put some of the resulting files in the ${\tt demo\_output}$ directory. Let's have a look at the inividual include regions to check how precise the fit was:

![Include results](https://drive.google.com/uc?export=view&id=19HAZLHzlF9UDqtuLnIf5UpiqrqOTNddd)

Everything looks reasonably good, eventhough it was run with relatively large tolerance values.

The final transmission model is stored in the ${\tt *\_TAC.fits}$ file, which we now have a look at.

In [ ]:
!ls molecfit_lecture/output_demo/
tac = fits.open('molecfit_lecture/output_demo/ESPRESSO_spectrum_TAC.fits')
print(tac.info())
print(tac[1].data.columns)
tac = tac[1].data

fig, axes = plt.subplots(5,1,figsize=(18,15))

[axes[i].plot(tac['WAVELENGTH'], tac['FLUX']/np.mean(tac['FLUX']), 'gray', lw=0.5, label='Original Spectrum') for i in range(len(axes))]
[axes[i].plot(tac['WAVELENGTH'], tac['mtrans'], 'r', alpha=0.5, lw=0.5, label='Transmission model' ) for i in range(len(axes))]
[axes[i].plot(tac['WAVELENGTH'], tac['tacflux']/np.mean(tac['tacflux']), 'k', lw=0.75, alpha=0.7, label='Corrected Spectrum') for i in range(len(axes))]

axes[0].set_ylim(-0.1, 2.2)
axes[0].legend(loc='upper left', fontsize=12)

axes[1].set_ylim(0., 1.8)
axes[1].set_xlim(5888, 5904)
axes[1].text(5888.5, 0.4, 'Na D doublet', fontsize=20)

axes[2].set_ylim(0., 1.8)
axes[2].set_xlim(6561, 6568)
axes[2].text(6561.3, 0.4, r'H$\alpha$', fontsize=20)

axes[3].set_ylim(0., 2.1)
axes[3].set_xlim(7160, 7300)
axes[3].text(7165, 0.2, r'H$_2$O band', fontsize=20)

axes[4].set_ylim(0., 2.1)
axes[4].set_xlim(6800, 7000)
axes[4].text(6808, 0.4, r'O$_2$ B band', fontsize=20)

And there you have it. We now have the telluric transmission model, as well the telluric corrected spectrum.  We will now quickly have a look at how to run the new version of ${\tt molecfit}$ through ${\tt esoreflex}$.  This is simply run by:

``` $> esoreflex molecfit ```